# Contour Tests

In [1]:
# Type imports
from typing import Any, Dict, List, Tuple

# Standard Libraries
from enum import Enum, auto
from pathlib import Path
from collections import Counter, defaultdict
from dataclasses import dataclass, field, asdict
from math import sqrt, pi, sin, cos, tan, radians
from math import isnan
from statistics import mean
from itertools import zip_longest

# Shared Packages
import pandas as pd
import numpy as np
import xlwings as xw

import pygraphviz as pgv
import networkx as nx
import matplotlib.pyplot as plt
import shapely


In [2]:
# Local modules
from types_and_classes import *
from utilities import *
from relations import *

In [3]:
%matplotlib inline

### Make test structures

In [25]:
# 6 cm x 6 cm box
box6 = shapely.Polygon(box_points(6))
slices_1 = make_contour_slices(roi_num=1, shape=box6, number_slices=5)

# 4 cm x 4 cm box
box4 = shapely.Polygon(box_points(4))
slices_2 = make_contour_slices(roi_num=2, shape=box4, number_slices=5)

# 6 cm x 6 cm box offset right by 3 cm
offset_box6 = shapely.Polygon(box_points(6, offset_x=3))
slices_3 = make_contour_slices(roi_num=3, shape=offset_box6, number_slices=5)

# 6 cm x 6 cm box with 4cm x 4 cm hole
box6 = shapely.Polygon(box_points(6))
box4 = shapely.Polygon(box_points(4))
slices = make_slice_list(5, start=0.2)
slices_4a = make_contour_slices(roi_num=4, shape=box6, number_slices=5,
                                offset_z=0.2)
slices_4b = make_contour_slices(roi_num=4, shape=box4, number_slices=5,
                                offset_z=0.2)

# 2 2x2 boxes
box2a = shapely.Polygon(box_points(2, offset_x=-3))
box2b = shapely.Polygon(box_points(2, offset_x=3))
slices_5a = make_contour_slices(roi_num=5, shape=box2a, number_slices=5)
slices_5b = make_contour_slices(roi_num=5, shape=box2b, number_slices=5)

# 2 cm x 2 cm box offset inf by 0.3 cm
slices_6 = make_contour_slices(roi_num=6, shape=offset_box6, number_slices=5,
                               offset_z=-0.3)

# Stacked Boxes
slices_7 = make_contour_slices(roi_num=7, shape=box4, height=4, spacing=0.5,
                               offset_z=2)
slices_8 = make_contour_slices(roi_num=8, shape=box4, height=4, spacing=0.5,
                               offset_z=-1.5)


# combine the slice data
slice_data = pd.concat([slices_1, slices_2, slices_3, slices_4a, slices_4b,
                        slices_5a, slices_5b, slices_6, slices_7, slices_8])
# convert slice data into a table of slices and structures
slice_table = make_slice_table(slice_data)

In [6]:
relationship_1_2 = Relationship((1, 2), slice_table)
print(relationship_1_2.relationship_type)

Relationship: Contains


In [7]:
relationship_1_2 = Relationship((1, 6), slice_table)
print(relationship_1_2.relationship_type)

Relationship: Overlaps


In [14]:
relationship_1_2 = Relationship((4, 2), slice_table)
print(relationship_1_2.relationship_type)

KeyError: '[4] not in index'

In [28]:
# Stacked Boxes
slices_7 = make_contour_slices(roi_num=7, shape=box4, height=4, spacing=0.5,
                               offset_z=2)
slices_8 = make_contour_slices(roi_num=8, shape=box4, height=4, spacing=0.5,
                               offset_z=-1.5)


# combine the slice data
slice_data = pd.concat([slices_7, slices_8])
# convert slice data into a table of slices and structures
slice_table = make_slice_table(slice_data)

In [29]:
slice_table.columns

Index([7, 8], dtype='int64', name='ROI Num')

In [30]:
structures = (7, 8)
relationship_1_2 = Relationship((7, 8), slice_table)
print(relationship_1_2.relationship_type)

Relationship: Overlaps


In [31]:
slice_structures = slice_table.loc[:, [structures[0], structures[1]]]
slice_structures

#### Horizontal Cylinders

![Horizontal Cylinders](./Images/FreeCAD%20Images/HorizontalCylinders.png)

In [9]:
slice_spacing = 0.1
# Body structure defines slices in use
body = make_box(roi_num=0, width=6, length=6, height=8, offset_z=-4,
                spacing=slice_spacing)

cylinder2h = make_horizontal_cylinder(radius=2, length=5, spacing=slice_spacing,
                                      roi_num=1)
cylinder1h = make_horizontal_cylinder(radius=1, length=5, spacing=slice_spacing,
                                      roi_num=2)
# combine the contours
slice_data = pd.concat([body, cylinder1h, cylinder2h])

# convert contour slice data into a table of slices and structures
slice_table = make_slice_table(slice_data, ignore_errors=True)


In [10]:
structures = tuple([1, 2])
relation = Relationship(structures, slice_table).relationship_type
print(relation)


Relationship: Partition


# ERROR: for Vertical Concentric Cylinders getting relationship *Contains*.  
- Should be *Partitions* if both cylinders end on the same slice  

 #### Vertical Concentric Cylinders
 
 ![Concentric Cylinders](./Images/FreeCAD%20Images/ConcentricCylinders.png)

In [11]:
slice_spacing = 0.5
# Body structure defines slices in use
body = make_vertical_cylinder(roi_num=0, radius=20, length=30, offset_z=-15,
                              spacing=slice_spacing)

cylinder6 = make_vertical_cylinder(roi_num=1, radius=6, length=10,
                                   spacing=slice_spacing)
cylinder4 = make_vertical_cylinder(roi_num=2, radius=4, length=10,
                                   spacing=slice_spacing)
# combine the contours
slice_data = pd.concat([body, cylinder6, cylinder4])

# convert contour slice data into a table of slices and structures
slice_table = make_slice_table(slice_data, ignore_errors=True)


In [12]:
relationship_1_2 = Relationship((1, 2), slice_table)
print(relationship_1_2.relationship_type)

Relationship: Contains
